In [1]:
import pandas as pd

In [2]:
auto_df = pd.read_csv("automatability.csv") #to transpose
relative_emp_df = pd.read_csv("relativeEmployment.csv") #to transpose
similar_df = pd.read_csv("newsimilarity.csv")
wagechange_df = pd.read_csv("wageChange.csv")

# all_csvs = [auto_df, relative_emp_df, similar_df, wagechange_df]
# for csv in all_csvs:
# auto_df.columns.values[0]='Job_Compared'
# relative_emp_df.columns.values[0]='Job_Compared'
# similar_df.columns.values[0]='Job_Compared'
# wagechange_df.columns.values[0]='Job_Compared'
            

In [3]:
auto_df = auto_df.transpose()
auto_df.columns = auto_df.iloc[0]
auto_df.drop(auto_df.index[0], inplace=True)

relative_emp_df = relative_emp_df.transpose()
relative_emp_df.columns = relative_emp_df.iloc[0]
relative_emp_df.drop(relative_emp_df.index[0], inplace=True)



# # auto_df=auto_df.set_index('Job Compared')
# # relative_emp_df=relative_emp_df.set_index('Job Compared')
# # similar_df=similar_df.set_index('Job Compared')
# # wagechange_df = wagechange_df.set_index('Job Compared')

In [4]:
# Creating an id/job name hash table

# temp_sim = pd.melt(similar_df, id_vars=['Job_Compared'], var_name='job_selected', value_name='similarity')
# temp_sim.columns =['job_compared','job_selected','similarity']
# temp_rec_therapist = temp_sim[temp_sim['job_selected']=='Recreational Therapists']
# temp_pre_export = temp_rec_therapist.reset_index()
# temp_pre_export = temp_pre_export[['index', 'Job_Compared']]
# temp_pre_export.columns=['id','job_name']
# temp_pre_export.to_csv("crosswalk.csv", index=False)

In [5]:
# Similarity - setting up data struture
temp_sim = pd.melt(similar_df, id_vars=['Job_Compared'], var_name='job_selected', value_name='similarity')
temp_sim.columns =['job_compared','job_selected','similarity']


# Intermediate step of setting up hash table for job name/id
# First we create the job hash table
temp_rec_therapist = temp_sim[temp_sim['job_selected']=='Recreational Therapists']
temp_pre_export = temp_rec_therapist.reset_index()
temp_pre_export = temp_pre_export[['index', 'job_compared']]
temp_pre_export.columns=['id','job_name']
# Next we load in the job descriptive stats and load join these to the hash table
jobs_descriptive = pd.read_csv("jobs.csv")
jobs_descriptive_hashed = pd.merge(jobs_descriptive, temp_pre_export, left_on = "Occupation", right_on = "job_name")
jobs_descriptive_hashed.columns=['to_delete', 'auto','wage','number','id','job_name']
jobs_descriptive_hashed = jobs_descriptive_hashed[['auto','wage','number','id','job_name']]
# Exporting crosswalk table
jobs_descriptive_hashed.to_csv("crosswalk.csv", index=False)


# Creating melted csv of selected job vs. compared job similarities, with IDs hashed
temp_sim = pd.merge(temp_sim,temp_pre_export, left_on='job_selected',right_on='job_name')
temp_sim.columns=['job_compared','to_delete_0','similarity', 'id_selected','to_delete_1']
temp_sim = temp_sim[['job_compared', 'similarity', 'id_selected']]

temp_sim = pd.merge(temp_sim,temp_pre_export, left_on='job_compared',right_on='job_name')
temp_sim.columns = ['to_delete_0','similarity','id_compared','id_selected','to_delete_1']
temp_sim = temp_sim[['similarity','id_compared','id_selected']]
temp_sim.to_csv("similarity.csv", index=False)

In [6]:
# Adding in skills crosswalk
skills_crosswalk = pd.read_csv("skills_raw.csv")
skills_crosswalk.columns.values[0]='skill_type'
skills_crosswalk.columns.values[1]='skill_name'

skills_crosswalk = pd.melt(skills_crosswalk, id_vars=['skill_type','skill_name'], var_name='job', value_name='imp')
skills_crosswalk = skills_crosswalk[skills_crosswalk['skill_type']!='ability']

all_skills = list(set(skills_crosswalk['skill_name'].tolist()))

skills_crosswalk = pd.DataFrame(all_skills)
skills_crosswalk['skill_id'] = range(0,68)
skills_crosswalk.columns=['skill','skill_id']
skills_crosswalk.to_csv("crosswalk_skills.csv", index=False)

In [7]:
# Adding in skills for each job
skills_raw = pd.read_csv("skills_raw.csv")
skills_raw.columns.values[0]='skill_type'
skills_raw.columns.values[1]='skill_name'
skills_raw

# Melting the data frame into a d3-friendly format
skills_edited = pd.melt(skills_raw, id_vars=['skill_type','skill_name'], var_name='job', value_name='imp')
skills_edited = skills_edited[skills_edited['skill_type']!='ability']
skills_edited = pd.merge(skills_edited, jobs_descriptive_hashed, left_on='job', right_on='job_name')

skills_edited = skills_edited[['id','skill_name','imp']]
skills_edited["rank"] = skills_edited.groupby("id")["imp"].rank("dense", ascending=False)
skills_edited=skills_edited[skills_edited['rank']<6]

# Sorting the skills df to get rid of the tied ranks
skills_edited = skills_edited.sort_values(by=['id','rank','skill_name'])
skills_edited['rank_final']=skills_edited.groupby("id")["imp"].rank("first", ascending=False)
skills_edited=skills_edited[skills_edited['rank_final']<6]

skills_edited.columns=['id_selected','skill','imp','to_delete','rank']
skills_edited=skills_edited[['id_selected','skill','imp','rank']]


# Joining skills for each profession to skills crosswalk
skills = pd.merge(skills_edited, skills_crosswalk, how='inner', left_on='skill', right_on='skill')
skills=skills[['id_selected','imp','skill_id','rank']]
skills = skills.sort_values(by=['id_selected','imp'])
skills.to_csv("skills.csv", index=False)

In [33]:
# Getting full skill list and importance for devs and truck trivers

# (Getting a column after it's been made into an index/can't be accessed in another way)
skills_list = pd.DataFrame(skills_raw.ix[:,1])

# Creating dev + trucker dF
dev_and_trucker_skills=skills_raw[['Software Developers, Applications','Heavy and Tractor-Trailer Truck Drivers']]
dev_and_trucker_skills['skill']=skills_list
# Renaming the columns in the data frame to reduce size
dev_and_trucker_skills.columns=['devs','truckers','skills']
dev_and_trucker_skills.to_csv("devs_and_truckers_skills.csv", index=False)



/Users/iblinderman/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [46]:
ten_profession_skills=skills_raw[['Choreographers','Dentists, General','Registered Nurses','Chiropractors','Farmers, Ranchers, and Other Agricultural Managers','Construction Managers','Firefighters','Geographers','Heavy and Tractor-Trailer Truck Drivers','Embalmers','Pipelayers']]
ten_profession_skills['skill']=skills_list
ten_profession_skills.columns=['choreographers','dentists','nurses','chiropractors','farmers','construction_managers','firefighters','geographers','truckers','embalmers','pipelayers','skills']

# Getting difference scores
ten_profession_skills['choreographers_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['choreographers'])
ten_profession_skills['dentists_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['dentists'])
ten_profession_skills['nurses_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['nurses'])
ten_profession_skills['chiropractors_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['chiropractors'])
ten_profession_skills['farmers_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['farmers'])
ten_profession_skills['construction_managers_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['construction_managers'])
ten_profession_skills['firefighters_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['firefighters'])
ten_profession_skills['geographers_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['geographers'])
ten_profession_skills['embalmers_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['embalmers'])
ten_profession_skills['pipelayers_difference']=abs(ten_profession_skills['truckers']-ten_profession_skills['pipelayers'])
ten_profession_skills.to_csv("ten_profession_skills.csv", index=False)

/Users/iblinderman/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/iblinderman/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/iblinderman/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

In [49]:
choreographer = ten_profession_skills[['choreographers','truckers','skills','choreographers_difference']]
dentists = ten_profession_skills[['dentists','truckers','skills','dentists_difference']]
nurses = ten_profession_skills[['nurses','truckers','skills','nurses_difference']]
chiropractors = ten_profession_skills[['chiropractors','truckers','skills','chiropractors_difference']]
farmers = ten_profession_skills[['farmers','truckers','skills','farmers_difference']]
construction_managers = ten_profession_skills[['construction_managers','truckers','skills','construction_managers_difference']]
firefighters = ten_profession_skills[['firefighters','truckers','skills','firefighters_difference']]
geographers = ten_profession_skills[['geographers','truckers','skills','geographers_difference']]
embalmers = ten_profession_skills[['embalmers','truckers','skills','embalmers_difference']]
pipelayers = ten_profession_skills[['pipelayers','truckers','skills','pipelayers_difference']]

choreographer.columns=['job_selected','job_compared','skills','difference']
dentists.columns=['job_selected','job_compared','skills','difference']
nurses.columns=['job_selected','job_compared','skills','difference']
chiropractors.columns=['job_selected','job_compared','skills','difference']
farmers.columns=['job_selected','job_compared','skills','difference']
construction_managers.columns=['job_selected','job_compared','skills','difference']
firefighters.columns=['job_selected','job_compared','skills','difference']
geographers.columns=['job_selected','job_compared','skills','difference']
embalmers.columns=['job_selected','job_compared','skills','difference']
pipelayers.columns=['job_selected','job_compared','skills','difference']

choreographer.to_csv('choreographer.csv', index=False)
dentists.to_csv('dentists.csv', index=False)
nurses.to_csv('nurses.csv', index=False)
chiropractors.to_csv('chiropractors.csv', index=False)
farmers.to_csv('farmers.csv', index=False)
construction_managers.to_csv('construction_managers.csv', index=False)
firefighters.to_csv('firefighters.csv', index=False)
geographers.to_csv('geographers.csv', index=False)
embalmers.to_csv('embalmers.csv', index=False)
pipelayers.to_csv('pipelayers.csv', index=False)


In [50]:

pipelayers
# relative_emp_df = relative_emp_df.reset_index()

# relative_emp_df.index.names = ['sdf']

# relative_emp_df
# df=df.rename(columns = {'two':'new_name'})


,job_selected,job_compared,skills,difference
0,0,3,Programming,3
1,23,32,Computers and Electronics,9
2,39,19,Engineering and Technology,20
3,25,25,Systems Analysis,0
4,10,0,Science,10
5,25,28,Systems Evaluation,3
6,55,15,Design,40
7,10,22,Operations Analysis,12
8,31,31,Fluency of Ideas,0
9,16,16,Technology Design,0


In [215]:
auto_df = auto_df[['Job_Compared','Heavy and Tractor-Trailer Truck Drivers','Athletes and Sports Competitors','Compensation and Benefits Managers','Construction Laborers']]
relative_emp_df = relative_emp_df[['Job_Compared','Heavy and Tractor-Trailer Truck Drivers','Athletes and Sports Competitors','Compensation and Benefits Managers','Construction Laborers']]
similar_df = similar_df[['Job_Compared','Heavy and Tractor-Trailer Truck Drivers','Athletes and Sports Competitors','Compensation and Benefits Managers','Construction Laborers']]
wagechange_df = wagechange_df[['Job_Compared','Heavy and Tractor-Trailer Truck Drivers','Athletes and Sports Competitors','Compensation and Benefits Managers','Construction Laborers']]

In [218]:
auto_df=auto_df.set_index('Job_Compared')
relative_emp_df=relative_emp_df.set_index('Job_Compared')
similar_df=similar_df.set_index('Job_Compared')
wagechange_df = wagechange_df.set_index('Job_Compared')

,Heavy and Tractor-Trailer Truck Drivers,Athletes and Sports Competitors,Compensation and Benefits Managers,Construction Laborers
Job_Compared,,,,
Recreational Therapists,-0.095455,0.737497,1.633326,-0.213737
"First-Line Supervisors of Mechanics, Installers, and Repairers",-0.346771,0.254758,0.901693,-0.432189
Emergency Management Directors,-0.441583,0.072636,0.625673,-0.514604
Mental Health and Substance Abuse Social Workers,-0.089599,0.748747,1.650376,-0.208647
Audiologists,-0.450246,0.055997,0.600454,-0.522134


In [221]:
auto_df = auto_df.add_prefix('auto_')
relative_emp_df = relative_emp_df.add_prefix('empl_')
similar_df = similar_df.add_prefix('similar_')
wagechange_df = wagechange_df.add_prefix('wage_')


In [223]:
auto_df= auto_df= auto_df.rename(columns=lambda x: x.replace(' ','_'))
relative_emp_df= relative_emp_df.rename(columns=lambda x: x.replace(' ','_'))
similar_df= similar_df.rename(columns=lambda x: x.replace(' ','_'))
wagechange_df= wagechange_df.rename(columns=lambda x: x.replace(' ','_'))

In [138]:
auto_df= auto_df= auto_df.rename(columns=lambda x: x.replace(' ','_'))
relative_emp_df= relative_emp_df.rename(columns=lambda x: x.replace(' ','_'))
similar_df= similar_df.rename(columns=lambda x: x.replace(' ','_'))
wagechange_df= wagechange_df.rename(columns=lambda x: x.replace(' ','_'))

In [234]:
truckers_wage = wagechange_df[wagechange_df.columns[0]].reset_index()
athletes_wage = wagechange_df[wagechange_df.columns[1]].reset_index()
comp_managers_wage = wagechange_df[wagechange_df.columns[2]].reset_index()
laborers_wage = wagechange_df[wagechange_df.columns[3]].reset_index()

    
truckers_similarity = similar_df[similar_df.columns[0]].reset_index()
athletes_similarity = similar_df[similar_df.columns[1]].reset_index()
comp_managers_similarity = similar_df[similar_df.columns[2]].reset_index()
laborers_similarity = similar_df[similar_df.columns[3]].reset_index()


truckers_relative_emp = relative_emp_df[relative_emp_df.columns[0]].reset_index()
athletes_relative_emp = relative_emp_df[relative_emp_df.columns[1]].reset_index()
comp_managers_relative_emp = relative_emp_df[relative_emp_df.columns[2]].reset_index()
laborers_relative_emp = relative_emp_df[relative_emp_df.columns[3]].reset_index()


truckers_automatability = auto_df[auto_df.columns[0]].reset_index()
athletes_automatability = auto_df[auto_df.columns[1]].reset_index()
comp_managers_automatability = auto_df[auto_df.columns[2]].reset_index()
laborers_automatability = auto_df[auto_df.columns[3]].reset_index()

In [248]:
dfs_trucker = [truckers_wage, truckers_similarity, truckers_relative_emp, truckers_automatability]

dfs_athlete = [athletes_wage,athletes_similarity,athletes_relative_emp,athletes_automatability]

dfs_comp_managers = [comp_managers_wage,comp_managers_similarity,comp_managers_relative_emp,comp_managers_automatability]

dfs_laborers = [laborers_wage,laborers_similarity,laborers_relative_emp,laborers_automatability]


df_trucker_final      =reduce(lambda left,right: pd.merge(left,right,on='Job_Compared'), dfs_trucker)
df_athlete_final      =reduce(lambda left,right: pd.merge(left,right,on='Job_Compared'), dfs_athlete)
df_comp_managers_final=reduce(lambda left,right: pd.merge(left,right,on='Job_Compared'), dfs_comp_managers)
df_laborers_final     =reduce(lambda left,right: pd.merge(left,right,on='Job_Compared'), dfs_laborers)


In [251]:
df_trucker_final = df_trucker_final.reset_index()
df_athlete_final = df_athlete_final.reset_index("Job_Compared")
df_comp_managers_final = df_comp_managers_final.reset_index("Job_Compared")
df_laborers_final = df_laborers_final.reset_index("Job_Compared")


In [253]:
raw_wage_auto_jobs = pd.read_csv("jobs.csv")
raw_wage_auto_jobs= raw_wage_auto_jobs
# .set_index("Occupation")


In [255]:
laborers = pd.merge(raw_wage_auto_jobs,df_laborers_final, right_on='Job_Compared',left_on='Occupation')
truckers = pd.merge(raw_wage_auto_jobs,df_trucker_final, right_on='Job_Compared',left_on='Occupation')
athletes = pd.merge(raw_wage_auto_jobs,df_athlete_final, right_on='Job_Compared',left_on='Occupation')
comp_managers = pd.merge(raw_wage_auto_jobs,df_comp_managers_final, right_on='Job_Compared',left_on='Occupation')

In [257]:
laborers.to_csv("laborers.csv")
truckers.to_csv("truckers.csv")
athletes.to_csv("athletes.csv")
comp_managers.to_csv("comp_managers.csv")

In [77]:
# auto_df.to_csv("auto.csv")
# relative_emp.to_csv("relative_emp.csv")
# similar_df.to_csv("simlar.csv")
# wagechange.to_csv("wage.csv")